In [64]:
import torch
from torch import nn
from torch.functional import F
from d2l import d2l
from IPython import display

In [98]:
train_iter, vocab = d2l.load_data_time_machine(32, 10, 'char')

In [99]:
class Model(nn.Module):
    def __init__(self, feature_size, hidden_size):
        super().__init__()
        self.lstm = nn.LSTM(feature_size, hidden_size, 1)
        self.linear = nn.Linear(hidden_size, feature_size)
        
    def forward(self, X, h, c):
        output, (h, c) = self.lstm(X, (h, c))
        return self.linear(output), (h, c)

In [100]:
hidden_size = 256
net = Model(len(vocab), hidden_size)

In [101]:
trainer = torch.optim.SGD(net.parameters(), lr=0.5)
loss = nn.CrossEntropyLoss()

In [102]:
import time
for epoch in range(500):
    h = torch.zeros(1, 32, hidden_size).to(torch.float32)
    c = torch.zeros(1, 32, hidden_size).to(torch.float32)
    for X, y in train_iter:

        h.detach_()
        c.detach_()

        X = F.one_hot(X.T, len(vocab)).to(torch.float32)
        y = F.one_hot(y.T, len(vocab)).to(torch.float32)    

        y_hat, (h, c) = net(X, h, c)

        l = loss(y_hat, y)
#         display.clear_output()
        print('epoch:', epoch + 1, l, end='\r')
        time.sleep(0.05)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    
    
    

KeyboardInterrupt: 